## Performance d'un tri fusion

Comparaison des tri_fusion : "plus grands devant" et "plus petits devant"

In [5]:
def fusion1(L1, L2):
    if L1 == []:
        return L2
    if L2 == []:
        return L1
    if L1[0] < L2[0]:  # plus petits devant
        return [ L1[0]] + fusion1(L1[1:], L2)
    else:
        return [ L2[0]] + fusion1(L1, L2[1:])
    
def tri_fusion1(L):
    nb = len(L)
    if nb <= 1:
        return L  
    return fusion1(tri_fusion1(L[:nb//2]), tri_fusion1(L[nb//2:]))

In [6]:
def fusion2(L1, L2):
    
    # plus grands devant
    
    
    

On compare les 2 algorithmes avec 

In [7]:
from numpy.random import permutation

# 4 listes permutées
for k in range(1, 5):
    print(list(permutation(2**k)))

[0, 1]
[1, 0, 2, 3]
[2, 7, 0, 3, 4, 5, 6, 1]
[5, 7, 8, 13, 4, 15, 10, 6, 0, 11, 2, 3, 14, 1, 9, 12]


In [8]:
for k in range(1, 5):
    %timeit tri_fusion1(list(permutation(2**k)))
    # timeit pour faire des stats sur le temps de calculs

4.55 µs ± 19.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.44 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
14.3 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
32 µs ± 575 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
for k in range(1,5):
    %timeit tri_fusion2(list(permutation(2**k)))

12.2 µs ± 32.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
21.6 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
30.6 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
61.2 µs ± 967 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
127 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
292 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
722 µs ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.78 ms ± 59.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.68 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Concaténer à gauche ou à droite sur une liste n'a pas la même complexité

En python, les listes sont des tableaux dynamiques.
On effectue des ré-allocation de mémoire pour agrandir le tableau, c'est à dire qu'il faut recréer une copie du tableau existant à un autre endroit physique de la mémoire !!

--> __Concaténation à gauche  == on change tous les indices, alors que concaténation à droite, on ajoute les indices...__


Selon l'implémentation des listes, on prendra ce qui a le moindre coût : mettre devant ou mettre derrière une liste, avoir accès au premier ou au dernier élément. 

## Parlons mémoire


Lorsque nous fusionnons les listes, on crée plein de petites listes intermédiaires (cf [pythontutor](http://pythontutor.com/visualize.html#code=def%20fusion%28xs%3A%20list,%20ys%3A%20list%29%20-%3E%20list%3A%0A%20%20%20%20%20%20%20%20if%20xs%20%3D%3D%20%5B%5D%20or%20ys%20%3D%3D%20%5B%5D%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20return%20xs%20%2B%20ys%0A%20%20%20%20%20%20%20%20elif%20xs%5B0%5D%20%3C%20ys%5B0%5D%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20return%20fusion%28xs%5B1%3A%5D,ys%29%20%2B%20%5Bxs%5B0%5D%5D%0A%20%20%20%20%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20return%20fusion%28xs,ys%5B1%3A%5D%29%20%2B%20%5Bys%5B0%5D%5D%0A%0Afusion%28%5B1,3,5%5D,%20%5B2,4,6%5D%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false))

On peut essayer de remédier à ce problème en utilisant un *buffer* (tampon).
On voudrait disposer d'un tampon dans lequel on va fusionner petit à petit les éléments du tableau à trier :

In [1]:
# Pour éviter les listes intermédiaires, le buffer

def fusion_en_place(L, tampon, lo, hi):
    mid = (lo + hi) // 2
    i, j = lo, mid # le tableau gauche s'arrête au milieu et celui de droite commence au milieu
    
    for pos in range(lo, hi):
        if j >= hi: # on a rangé toute la partie droite
            tampon[pos] = L[i] # on met le prochain élément de la partie gauche restante
            i += 1 # on avance dans la partie gauche d'un cran
            
        elif i >= mid: # on a rangé toute la partie gauche
            tampon[pos] = L[j] # on met met le prochain élément de la partie droite restante
            j += 1  # on avance dans la partie droite d'un cran
            
        elif L[i] <= L[j]: # il en reste des 2 côtés : qui est le plus petit ?
            tampon[pos] = L[i] # on place le plus petit
            i += 1 # on avance dans la partie gauche d'un cran
            
        else: 
            tampon[pos] = L[j]
            j += 1 # on a placé xs[j] donc on avance à droite
            
    for pos in range(lo, hi):
        L[pos] = tampon[pos] # xs trié entre les indices lo et hi

def tri_fusion_en_place(L):
    def aux(tampon, lo, hi):
        if hi - lo >= 2:
            mid = (lo + hi) // 2
            aux(tampon, lo, mid)
            aux(tampon, mid, hi)
            fusion_en_place(L, tampon, lo, hi)
    aux([0]*len(L), 0, len(L))

L = [1, 5, 2, 4, 9, 3, 2, 5, 47, 56, 12, 23, 65, 54]
print(L)
tri_fusion_en_place(L)
print(L)

[1, 5, 2, 4, 9, 3, 2, 5, 47, 56, 12, 23, 65, 54]
[1, 2, 2, 3, 4, 5, 5, 9, 12, 23, 47, 54, 56, 65]


In [ ]:
import os
os.sys.setrecursionlimit(100000)

# afficher les timeit



Comparaispon graphique :

In [1]:
from matplotlib import pyplot as plt
from time import perf_counter
import os
os.sys.setrecursionlimit(100000)

def temps(tri):
    n = []
    temps =[]
    for k in range(1, 12):
        L = list(permutation(2**k))
        n.append(len(L))
        start = perf_counter()
        tri(L)
        stop = perf_counter()
        temps.append(stop - start)
    return n, temps

#graphique



NameError: name 'tri_fusion1' is not defined

# Les tris en Python

Python a deux fonctions permettant de trier des collections : `sorted` et `sort`. La première effectue un tri sur un `iterable` et crée une *nouvelle* liste sans modifier l'itérable donné en argument.  
`sort` au contraire est une *méthode* de la classe `list` et effectue un tri *en place* de la liste de départ : celle-ci est donc modifiée par une suite d'opérations à effet de bord. Dans les premières versions de Python, ces fonctions étaient dérivées du tri rapide mais finalement l'algorithme utilisé actuellement est dérivé du tri fusion (il s'appelle [`timsort`](https://fr.wikipedia.org/wiki/Timsort) du nom de son auteur). 
Il est ainsi important de noter que l'idée du tri fusion de John Von Neumann est toujours utilisée. Son introduction dans les années 50 a marqué lémergence de l'analyse des algorithmes dont plus tard Don Knuth sera le héraut. C'est aussi l'exemple phare du principe de *diviser pour régner* qui sera l'objet du prochain chapitre...

# Illustrations

Voici deux illustrations du tri fusion : 


<figure class="video_container">
  <video controls="true" allowfullscreen="true" autoplay="true">
    <source src="merge.mp4" type="video/mp4">
  </video>
</figure>

[illustration du tri fusion...à coder](https://i.imgur.com/MAusQ25.mp4)


[et bien d'autres tris](https://imgur.com/gallery/voutF)